In [1]:
# Usage guide:

# This document is not kept on record. If the calcualtions are to be kept,
# copy the whole notebook and put it in "quick_calculations" or appropriate
# folders.

In [2]:
# import all the E9 stuff
import logging
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(1,
    "C:\\Users\\ken92\\Documents\\Studies\\E5\\simulation\\E9_simulations")
import E9_fn.E9_constants as E9c
import E9_fn.E9_numbers as E9n
import E9_fn.E9_cooling as cool
import E9_fn.polarizabilities_calculation as E9pol
from E9_fn import util

# Logging
logpath = '' # '' if not logging to a file
loglevel = logging.INFO
logroot = logging.getLogger()
list(map(logroot.removeHandler, logroot.handlers))
list(map(logroot.removeFilter, logroot.filters))
logging.basicConfig(filename = logpath, level = loglevel)

c:\Users\ken92\anaconda3\envs\spyder-env\Lib\site-packages\gftool\precision.py:16: UserWarning: No quad precision datatypes available!
Some functions might be less accurate.
  warnings.warn("No quad precision datatypes available!\n"


In [3]:
# input parameters
lambda_DMD = 638e-9     # wavelength used for DMD potential
pwr_at_atom = 1         # power at atoms, in watts
FoV_at_atom = 200e-6**2 # field of view (region illuminated) by the DMD

I_at_atom = pwr_at_atom / FoV_at_atom
f_DMD = E9c.c_light / lambda_DMD
V_DMD = E9n.V0_from_I(E9c.gamma_40K_D2, (E9c.nu_K40_4_2P3o2 + E9c.nu_K40_4_2P1o2)/2,
                      f_DMD, I_at_atom, E9n.K40_4_2S1o2_F9o2.gF, 9/2)
print("V_DMD = {} kHz".format(V_DMD / E9c.hnobar / 1000))

# Polarizibility based calculation
s_pol = E9pol.alpha_pol(0, lambda_DMD, E9pol.K_4S1o2_LOI, '4S1o2')
print("V_DMD = {} kHz (from polarizibility)".format(- s_pol * I_at_atom * E9c.Z_0 / 2 / E9c.hnobar / 1000))

V_DMD = 82.09433987605135 kHz
K_4S1o2_3D5o2 transition does not have f_ik data (not E1 allowed?)
K_4S1o2_3D5o2 transition does not have f_ik data (not E1 allowed?)
K_4S1o2_4D5o2 transition does not have f_ik data (not E1 allowed?)
K_4S1o2_4D3o2 transition does not have f_ik data (not E1 allowed?)
V_DMD = 73.9405425702114 kHz (from polarizibility)


In [7]:
w_MOT = 6e-3
I_MOT = 60e-3
gamma_Yb174 = 182.4e3 * 2 * np.pi
lambda_Yb174 = 555.8e-9
m_Yb174 = 174 * E9c.m_u
mu_Yb174 = 1.5 * E9c.mu_B
v_MOT_actual = 5

k_MOT = 2 * np.pi / lambda_Yb174
v_MOT_max = np.sqrt(2 * E9c.hbar * k_MOT * gamma_Yb174 * w_MOT / m_Yb174)
A_max = (E9c.hbar * k_MOT)**2 / (2 * m_Yb174) * gamma_Yb174 / (mu_Yb174 * v_MOT_actual)
print(v_MOT_max)
print("A_max = {} G/cm".format(A_max * 100))

7.532905935793225
A_max = 4.052495215875869 G/cm
